In [22]:
import requests, re, urllib, itertools
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse
from collections import Counter, Iterable
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from anytree import Node
from nltk.tree import Tree
from nltk.chunk.regexp import RegexpParser
from nltk import pos_tag, word_tokenize

def checkAppropriateURL(url):
    parsedURL = urlparse(url)   

    extn = parsedURL.path.split(".")[-1]
    site = parsedURL.netloc.split(".")
    path = parsedURL.path.split("/")[-1]

    if extn in ["pdf","doc","docx","xls","xlsx","ppt","pptx","odt"]:
        return False
    if any(i in site for i in ["pinterest"]):
        return False
    r = dict(requests.head(url).headers)["Content-Type"].split(";")[0]
    if(not r=="text/html"):
        return False
    return True

def scrapeGoogleForAbstracts(query, count):
    words = query.split()
    q = ("+").join(words)
    print (q)
    query_url_goog = "https://www.google.co.in/search?q=\"" + q + "\""
    data = requests.get(query_url_goog).text
    soup = BeautifulSoup(data, "lxml")
    h3Rows = soup.find_all("h3", {"class":"r"})
    spanRows = soup.find_all("span", {"class":"st"})
    zippedRows = zip(h3Rows,spanRows)
    urls = []
    print (list(zippedRows))
    for zippedRow in zippedRows:
        try:
            h3Row = zippedRow[0]
            url = h3Row.find("a")['href']
            if(url.split("?")[0]=="/search"):
                continue
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        par = urllib.parse.parse_qs(urlparse(url).query)
        try:
            appendingURL = par['q'][0]
        except:
            continue
        if(not checkAppropriateURL(appendingURL)):
            continue
        matchedQuery = query
        allbElems = zippedRow[1].findAll('b')
        for bElem in allbElems:
            if bElem.string[0].isalnum():
                matchedQuery = bElem.string
        urls.append((matchedQuery,appendingURL))
    i = 1
    same = 0
    prev = 0
    while(len(urls)<count):
#         print ("Query: ",query)
        tempurl = query_url_goog + "&start=" +str(i*10)
        i+=1
        data = requests.get(tempurl).text
        soup = BeautifulSoup(data, "lxml")
        h3Rows = soup.find_all("h3", {"class":"r"})
        spanRows = soup.find_all("span", {"class":"st"})
        zippedRows = zip(h3Rows,spanRows)
        for (j,zippedRow) in enumerate(zippedRows):
            try:
                h3Row = zippedRow[0]
                url = h3Row.find("a")['href']
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
            par = urllib.parse.parse_qs(urlparse(url).query)
            try:
                appendingURL = par['q'][0]
                if(not checkAppropriateURL(appendingURL)):
                    continue
                matchedQuery = query
                allbElems = zippedRow[1].findAll('b')
                for bElem in allbElems:
                    if bElem.string[0].isalnum():
                        matchedQuery = bElem.string
                urls.append((matchedQuery, appendingURL))
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
        if prev == len(urls):
            same +=1 
            if same>=10:
                same = 0
                break
        prev = len(urls)
    return urls[:count]

In [7]:


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getTextFromURL(url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
    html = urllib.request.urlopen(urllib.request.Request(url, headers=hdr))
    inputText = text_from_html(html)
#     print (inputText)
    return inputText

def getAbstracts(query, count):
    urls = scrapeGoogleForAbstracts(query, count)
    print (urls)
    print ("lenght",len(urls))
    texts = []
    for (query, url) in urls:
        print (url)
        try:
            text = getTextFromURL(url)
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        texts.append((query,url,text.strip()))
    return texts


In [8]:
def processWordSet(wordSet):
    for (i,word) in enumerate(wordSet):
        wordSet[i] = re.sub("^[^a-zA-Z]*|[^a-zA-Z(?<!(s\'))]*$","",word)
    return wordSet

def getHypernymsFromWordnet(word):
    allSynsets = wn.synsets(word)
    multiList = [[[a.name() for a in b.lemmas()] for b in c.hypernyms()]for c in wn.synsets(word)]
    finalList = flatten(multiList)
    return Counter(finalList)

def compareTuple(x,model):
    try:
        sim = model.similarity(x[0],x[1])
        return sim
    except:
        return 0

def initNodesList(wordSet):
    nodesList = []
    for wordNode in wordSet:
        nodesList.append(Node(wordNode))
    return nodesList

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

# file = open("pizza.txt")
# text = file.read()
# wordSet = processWordSet(list(set(text.split())))
# wordSet = list(set(filter(None, wordSet)))

wordSet = ["pizzas","pizza toppings"]

model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

# Sort by decreasing similarity
tuplesList = list(itertools.combinations(wordSet,2))
sortedList = sorted(tuplesList, key=lambda x : compareTuple(x,model), reverse=True)

clusteredSet = set()
hypernymDict = {}

root = Node("root")
nodesList = initNodesList(wordSet)


/usr/local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
def getHearstPatterns(word):
    # Pattern, Instances after/before, One/many instances
    patterns = []
    patterns.append((word + " such as", True, True))
    patterns.append(("such " + word + " as", True, True))
    patterns.append(("and other " + word, False, True))
    patterns.append(("or other " + word, False, True))
    patterns.append((word + ", including", True, True))
    patterns.append((word + ", especially", True, True))
    return patterns

def getInstances(text):
    grammar = """
        PRE:   {<NNS|NNP|NN|NP|JJ|UH>+}
        MID: {<DT|IN|POS|FW|-|NP|NPS|NN|NNS>+}
        INSTANCE:   {(<DT+>)?(<JJ+>)?<PRE>(<MID><PRE>)?}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "INSTANCE"):
            # print (i.leaves())
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

def extractHypernyms(posList, text, query):
    allInstances = []
    for (st,end) in posList:
        if query[2]:
            if query[1]:
                #Instances after, and exists                
                tempText = text[end+1:]
                rest = re.match(".*?\.",tempText)
                if rest:
                    rest = rest.group()
                else:
                    rest = tempText
                andExists = re.search(" (and|or) ", rest)
                if andExists:
                    print ("rest:", rest)
                    lastConceptPos = andExists.end()
                    lastConcept = rest[lastConceptPos:]
                    print ("lastConcept: ", lastConcept)
                    lastConceptExtracted = getInstances(lastConcept.strip())
                    if lastConceptExtracted:
                        allInstances.append(lastConceptExtracted[0])
                    prevConceptsEnd = andExists.start()
                    prevConcepts = rest[:prevConceptsEnd]
                    prevConceptsList = prevConcepts.split(",")
                    prevConceptsList = [a for a in prevConceptsList if a.strip()]
                    print ("****START")
                    print ("previous Concepts:",prevConcepts)
                    for concept in prevConceptsList:
                            print (concept.strip(), getInstances(concept.strip()))
                            conceptExtracted = getInstances(concept.strip())
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[-1])
                    print ("*****DONE")
                else:
                    allConcepts = rest.split(",")
                    print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
                    for (i,concept) in enumerate(allConcepts):
                            if i==(len(allConcepts)-1):
                                print (concept)
                                conceptExtracted = getInstances(concept.strip())
                                print ("1######CONCEPT EXTRACTED:", conceptExtracted)
                                if conceptExtracted:
                                    allInstances.append(conceptExtracted[0])
                            else:
                                conceptExtracted = getInstances(concept.strip())
                                print ("######CONCEPT EXTRACTED:", conceptExtracted)
                                if conceptExtracted:
                                    allInstances.append(conceptExtracted[-1])
            else:
                tempText = text[:st]
                print ("tempText", tempText)
                try:
                    lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                    lastSent = tempText[:lastSentPos]
                except:
                    lastSent = tempText
                lastSentList = lastSent.split(",")
                lastSentList = [a for a in lastSentList if a.strip()]
                if len(lastSentList) == 1:
                    conceptExtracted = getInstances(lastSentList[0].strip())
                    if conceptExtracted:
                        allInstances.append(conceptExtracted[-1])
                else:
                    for (i,concept) in enumerate(lastSentList):
                        conceptExtracted = getInstances(concept.strip())
                        if conceptExtracted:
                            if i==(len(lastSentList)-1):
                                allInstances.append(conceptExtracted[0])
                            else:
                                allInstances.append(conceptExtracted[-1])
        else:
            tempText = text[:st]
            try:
                lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                lastSent = tempText[:lastSentPos]
            except:
                lastSent = tempText
            lastConceptExtracted = getInstances(lastSent.strip())
            if lastConceptExtracted:
                allInstances.append(lastConceptExtracted[-1])
    return allInstances

def getHypernymsFromWWW(word):
    allHypernyms = []
    queriesList = getHearstPatterns(word)
    for query in queriesList:
        texts = getAbstracts(query[0], 10)
        for (q,url,text) in texts:
            startEndPositions = []
            try:
                for m in re.finditer(re.escape(q), text):
                    try:
                        startEndPositions.append((m.start(), m.end()))
                    except:
                        print ("query:",q,"url:",url)
                        continue
            except:
                print ("1","query:",q,"url:",url)
                continue
            allHypernyms.extend(extractHypernyms(startEndPositions, text, query))
    return Counter(allHypernyms)

def getHypernyms(word):
    wordnet = getHypernymsFromWordnet(word)
    www = getHypernymsFromWWW(word)
    return (wordnet + www)
        

In [20]:
ls = {}
ls["dict"] = Counter([1,2,3]) + Counter([1,3,6])
ls

{'dict': Counter({1: 2, 2: 1, 3: 2, 6: 1})}

In [20]:
for word in wordSet:
    hypernymDict[word] = getHypernyms(word)
    

pizzas+such+as
[(<h3 class="r"><a href="/url?q=https://en.wikipedia.org/wiki/Pizza&amp;sa=U&amp;ved=0ahUKEwjkmpfSwNHfAhWNfn0KHeQLBGkQFggYMAE&amp;usg=AOvVaw24iKXJAWr2RqM8leqKIYGh">Pizza - Wikipedia</a></h3>, <span class="st"></span>), (<h3 class="r"><a href="/url?q=https://www.telepizza.com/en/&amp;sa=U&amp;ved=0ahUKEwjkmpfSwNHfAhWNfn0KHeQLBGkQFggcMAI&amp;usg=AOvVaw1iXWOBMt1kmo8hURVR9opj">telepizza.com</a></h3>, <span class="st">From classics such as Barbecue or Carbonara to the more innovative <b>pizzas</b> <br/>
<b>such as</b> Natura or Wok. Discover our finest pizzas. Our products. 120. 13. 90. 26.</span>), (<h3 class="r"><a href="/url?q=https://en.wikipedia.org/wiki/Pizza&amp;sa=U&amp;ved=0ahUKEwjkmpfSwNHfAhWNfn0KHeQLBGkQFgghMAM&amp;usg=AOvVaw26dYHbXCsYrEGRW2iGO7Hk">Pizza - Wikipedia</a></h3>, <span class="st">Pizza is a savory dish of Italian origin, consisting of a usually round, flattened <br/>
base of leavened ..... There are several instances of more expensive <b>pizzas, such</

****START
previous Concepts: the 
Sicilian-style Burratina Di Margherita (Burrata cheese, cherry tomatoes 
tossed with basil, extra virgin olive oil
the 
Sicilian-style Burratina Di Margherita (Burrata cheese ['Sicilian-style Burratina Di Margherita', 'Burrata cheese']
cherry tomatoes 
tossed with basil ['cherry tomatoes', 'basil']
extra virgin olive oil ['extra virgin olive oil']
*****DONE
rest: a Reuben Pizza with shredded corned beef or shaved pastrami, cheese, onions and sauerkraut.
lastConcept:  shaved pastrami, cheese, onions and sauerkraut.
****START
previous Concepts: a Reuben Pizza with shredded corned beef
a Reuben Pizza with shredded corned beef ['Reuben Pizza with shredded', 'beef']
*****DONE
rest: the “Green Monster” with pesto, spinach, artichoke hearts, goat cheese and roasted red peppers.
lastConcept:  roasted red peppers.
****START
previous Concepts: the “Green Monster” with pesto, spinach, artichoke hearts, goat cheese
the “Green Monster” with pesto ['“ Green Monster 

[(<h3 class="r"><a href="/url?q=https://www.tripadvisor.com/LocationPhotoDirectLink-g187147-d742407-i122506835-Pizza_Vesuvio-Paris_Ile_de_France.html&amp;sa=U&amp;ved=0ahUKEwjCi4SpwdHfAhWOT30KHR8uD90QFggUMAA&amp;usg=AOvVaw0lpe4oKL1yMLjerYIM__T_">4 Stagione pizza <b>and other pizzas</b> - Picture of Pizza Vesuvio, Paris ...</a></h3>, <span class="st">Pizza Vesuvio, Paris Picture: 4 Stagione pizza <b>and other pizzas</b> - Check out <br/>
TripAdvisor members' 57576 candid photos and videos of Pizza Vesuvio.</span>), (<h3 class="r"><a href="/url?q=http://www.pizzamiastoneham.com/pizza-nearby/&amp;sa=U&amp;ved=0ahUKEwjCi4SpwdHfAhWOT30KHR8uD90QFggZMAE&amp;usg=AOvVaw1RtRb98SkMp6RcVDTwZ3Y5">Pizza Nearby | Pizza Mia</a></h3>, <span class="st">Some of the many options include Mexican, Californian, Cheese, BBQ Chicken, <br/>
Vegetarian <b>and other pizzas</b>. Quick delivery at your place. We are also trusted ...</span>), (<h3 class="r"><a href="/url?q=https://www.cnbctv18.com/economy/here-are-t

tempText Bite into homely pizzas                    Home Bengaluru  Top Stories  Crime  Politics  Infrastructure  Life in the City Karnataka  Top Stories  Politics  Districts National  Politics  North and Central  South  East and Northeast  West Sports  Cricket  Football  Tennis  Other Sports  Sportscene Business  Business News  Economy & Business  Family Finance  Technology  DH Wheels Videos  Bengaluru  Karnataka  People  Popular Opinion  Editorials  Panorama  Comment  In Perspective  Right in the Middle Features  Metrolife  Travel  Spectrum  Sunday Herald  Living  DH Education  19 to Watch in '19                       Settings   Logout     Sign in    With Facebook  With Google        Please fill the details to complete Sign In    Email:      Birthday:         Gender:   Choose gender  Male  Female     Location:   Choose Location  Tumakuru  Belagavi  Bagalkot  Vijayapura  Dharwad  Gadag  Haveri  Uttarakannada  Bengaluru Urban  Bengaluru Rural  Chikkaballapura  Chithradurga  Davanagere 

tempText Windhoek     Windhoek Tourism    Windhoek Hotels    Windhoek Guest House    Windhoek Holiday Homes    Windhoek Holiday Packages    Windhoek Flights    Windhoek Restaurants    Windhoek Attractions    Windhoek Travel Forum    Windhoek Photos    Windhoek Map    Windhoek Visitors Guide        All Windhoek Hotels    Windhoek Hotel Deals    Last Minute Hotels in Windhoek    By Hotel Type    Business Hotels in Windhoek    Spa Resorts in Windhoek    Luxury Hotels in Windhoek    Family Hotels in Windhoek    Casinos in Windhoek    Romantic Hotels in Windhoek      By Hotel Class    5-star Hotels in Windhoek    4-star Hotels in Windhoek    3-star Hotels in Windhoek      Popular Amenities    Windhoek Hotels with a Pool    Windhoek Hotels with Parking      Popular Windhoek Categories    Hotels with Bars in Windhoek    Hotels with Room Service in Windhoek    Windhoek Hotels with Shuttle    Quiet Hotels in Windhoek    Downtown Hotels in Windhoek    Safe Hotels in Windhoek    Windhoek Hotels w

...

tempText Advertisements              About         jovina cooks      Healthy Mediterranean Cooking at Home           What Is the Difference Between Calzone and Stromboli?  July 31, 2012         Calzone  Calzone is a turnover with ingredients similar to pizza. The making of calzones started in Naples, Italy in the 18th century. The name came from the baggy pants worn by men during the time. The ingredients of calzones usually consist of mozzarella, ricotta, tomato sauce, 
or+other+pizza+toppings
[(<h3 class="r"><a href="/url?q=https://www.soulfullymade.com/pizza-noodle-bake/&amp;sa=U&amp;ved=0ahUKEwja8vX9xdHfAhVBU30KHb9GAJUQFggYMAE&amp;usg=AOvVaw32MiIYLWRfNb467e_d50zy">Pizza Noodle Bake - Soulfully Made</a></h3>, <span class="st"></span>), (<h3 class="r"><a href="/url?q=https://www.kiwico.com/diy/explore/pepperoni-or-other-pizza-toppings&amp;sa=U&amp;ved=0ahUKEwja8vX9xdHfAhVBU30KHb9GAJUQFggcMAI&amp;usg=AOvVaw30V9xwC5imM-O_Pj6Zq8bG">pepperoni <b>or other pizza toppings</b></a></h3>, <spa

tempText CALCULATE THE COST OF QUALITY INGREDIENTS What impact do higher quality ingredients have on your profit margin? Are you considering a change in your sausage, beef 
tempText Home About Me Contact Me                        Recipes Recipe Index All the Mrs’s recipes Appetizers Main Dish Sides Desserts Breakfast Bread Southern Classics Healthy Eating Beverages Household All Household Make Your Own DIY Travel Shop      Apr  07 Pizza Swirl Buns Posted by Crystal //  0 Comments »  Pizza is one of those feel good foods that almost everyone likes.  What’s not to like? The crust, the sauce, the pepperoni – and of course, the cheese. Everything about it is perfect. Since it is such a perfect food, how much better would it be if you stuffed the 
tempText Home About Me Contact Me                        Recipes Recipe Index All the Mrs’s recipes Appetizers Main Dish Sides Desserts Breakfast Bread Southern Classics Healthy Eating Beverages Household All Household Make Your Own DIY Travel Sho

tempText Finger Lakes                               Toggle navigation                       Food Near You    Food Near You Overview   Farmers Markets   School & Community Gardens   Free Meals for Youth   Food Pantries / Soup Kitchens   Lookup     Recipes   Events    Tips    Fruits, Veggies, and More   Grab the Good Stuff   School and Community Gardens   Physical Activity   Healthy Eating   Rethink Your Drink   Balance Calories   Smart Shopping   School Wellness      About Us    About   In the News   Newsletters   Stories   Staff   FAQ   Contact Us         Subscribe          Subscribe to our Newsletter                      CREDIT: Kim Siciliano              ← Home           My Personal Pizza  Source:  Modified from a Food Hero recipe   Serving size: 1/2 half English muffin  Servings per recipe: one personal pizza  This is a quick and easy after school snack for kids of all ages!  Print Recipe  View Nutrition Facts                            Ingredients   1/2 whole wheat English muffin 1

pizza+toppings,+including
[(<h3 class="r"><a href="/url?q=https://www.pizzaironside.com/vegetarian-pizza-recipes/&amp;sa=U&amp;ved=0ahUKEwjB2LqixtHfAhVaVH0KHeQoAFIQFggdMAI&amp;usg=AOvVaw0PV_SoS-xrgsi9Ds5Ebwvw">12 Best Vegetarian Pizza Recipes and Toppings | Ironside Kitchen</a></h3>, <span class="st"></span>), (<h3 class="r"><a href="/url?q=https://www.pizzaironside.com/vegetarian-pizza-recipes/&amp;sa=U&amp;ved=0ahUKEwjB2LqixtHfAhVaVH0KHeQoAFIQFgghMAM&amp;usg=AOvVaw22Mcpg0DnxiYxdqrJ-BXfU">12 Best Vegetarian Pizza Recipes and Toppings | Ironside Kitchen</a></h3>, <span class="st">24 May 2017 <b>...</b> 12 Best Vegetarian Pizza Recipes and Toppings. Mediterranean Pizza with <b>Pesto</b> and <b>Artichokes</b>. Vegan “<b>sausage</b>” and <b>cheese</b> pizza. Gourmet Vegetarian <b>Pumpkin</b> and White <b>Sauce</b> Pizza. Gourmet Savory Apple and White <b>Cheddar</b> Pizza. Grilled <b>eggplant</b> and fresh <b>basil</b> pizza. Vegan pizza bagels. Vegetarian <b>Barbecue</b> “Chicken Wing” .

[(<h3 class="r"><a href="/url?q=https://www.pritikin.com/your-health/healthy-living/eating-right/1348-pritikin-pizza-party.html&amp;sa=U&amp;ved=0ahUKEwjiyaK4xtHfAhWDV30KHYAHAhoQFggcMAI&amp;usg=AOvVaw0AGyN8GuOcUbwMqQC70A6p">Healthy Pizza Recipes | Pritikin, a Weight-Loss Resort</a></h3>, <span class="st"></span>), (<h3 class="r"><a href="/url?q=https://spoonuniversity.com/recipe/10-grilled-pizza-recipes&amp;sa=U&amp;ved=0ahUKEwjiyaK4xtHfAhWDV30KHYAHAhoQFgggMAM&amp;usg=AOvVaw0wPYLqFSOFpa3xOtPd52uf">10 Grilled Pizza Recipes That Are Perfect for Summer Barbecues</a></h3>, <span class="st">Sausage is one of my absolute favourite <b>pizza toppings, especially</b> if it's a <br/>
sausage with a little more heat to it. I personally prefer chorizo, but pretty much <br/>
any ...</span>), (<h3 class="r"><a href="/search?q=%22pizza+toppings,+especially%22&amp;ie=UTF-8&amp;prmd=ivns&amp;tbm=isch&amp;tbo=u&amp;source=univ&amp;sa=X&amp;ved=0ahUKEwjiyaK4xtHfAhWDV30KHYAHAhoQsAQIJQ">Images for <b>"pizz

In [21]:
# finalDict = {}
# for hypernym in hypernymDict:
#     finalDict[hypernym] = {}
#     for word in hypernymDict[hypernym]:
#         if hypernymDict[hypernym][word] > 1:
#             finalDict[hypernym][word] = hypernymDict[hypernym][word]
# finalDict

hypernymDict

{'pizzas': Counter({'dish': 1,
          'crispy cheese': 1,
          'thin': 1,
          'Valaisanne': 1,
          'Vaudoise': 1,
          'Barack Obama': 1,
          'François Hollande': 1,
          'Thai Dye pizza': 1,
          'Caribbean-themed pizzas': 1,
          'such as the Whitewater': 1,
          'Wis': 2,
          'Whitewater': 1,
          'onion': 1,
          'sausage': 4,
          'balsamic reduction': 2,
          'Burrata cheese': 2,
          'basil': 2,
          'extra virgin olive oil': 2,
          'pastrami': 1,
          'beef': 1,
          'red peppers': 1,
          '“ Green Monster ” with pesto': 1,
          'spinach': 2,
          'artichoke hearts': 1,
          'goat cheese': 1,
          'pesto-chicken Green Stripe': 1,
          'restaurant owners': 1,
          'hoteliers': 1,
          'tortilla chips': 3,
          'Taco Joe': 3,
          'taco-seasoned ground beef': 3,
          'lettuce': 3,
          'tomatoes': 3,
          'cheeses'